# scikit learning

https://youtu.be/t4319ffzRg0?list=PLQVvvaa0QuDc-3szzjeP6N6b0aDrrKyL-


http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
scikit-learn cheat-sheet

In [1]:
import quandl;
import pandas as pd;

import pickle;

import matplotlib.pyplot as plt;
from matplotlib import style;
style.use("ggplot");

import numpy as np;

from statistics import mean;

from sklearn import svm, preprocessing, cross_validation;

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
api_key = open("quandlapikey.txt", "r").read();

def mortgage_30y_resampled():
    df = quandl.get("FMAC/MORTG", trim_start = "1975-01-01", authtoken = api_key);       
    df["Value"] = (df["Value"] - df["Value"][0]) / df["Value"][0] * 100.0;
    df.columns = ["M30"];
    return df.resample("M").mean();

def state_list():
    fiddy_states = pd.read_html("https://simple.wikipedia.org/wiki/List_of_U.S._states");
    return fiddy_states[0][0][1:];

def grap_initial_state_data_start_pct():
    states = state_list();
    main_df = pd.DataFrame();
    for ab in states:
        querry = "FMAC/HPI_" + ab;
        df = quandl.get(querry, authtoken = api_key);
        df.columns = [ab];        
        df[ab] = (df[ab] - df[ab][0]) / df[ab][0] * 100.0; # <-------
        if main_df.empty:
            main_df = df;
        else:
            main_df = main_df.join(df);

    pickle_out = open("./data/fiddy_states.pickle", "wb");
    pickle.dump(main_df, pickle_out);
    pickle_out.close();
    
def HPI_Benchmark():
    df = quandl.get("FMAC/HPI_USA", authtoken = api_key);
    df.columns = ["US"];        
    df["US"] = (df["US"] - df["US"][0]) / df["US"][0] * 100.0; # <-------
    return df;


In [3]:
def sp500_data():
    df = quandl.get("YAHOO/INDEX_GSPC", trim_start = "1975-01-01", authtoken = api_key);
    df["Adjusted Close"] = (df["Adjusted Close"] - df["Adjusted Close"][0]) / df["Adjusted Close"][0] * 100.0; # <-------
    df = df.resample("M").mean();
    df.rename(columns={"Adjusted Close":"sp500"}, inplace = True);
    df = df["sp500"];
    return df;

In [4]:
df = sp500_data();
print(df.head());

Date
1975-01-31     3.323491
1975-02-28    14.049322
1975-03-31    19.367785
1975-04-30    20.636734
1975-05-31    28.287322
Freq: M, Name: sp500, dtype: float64


In [5]:
def gdp_data():
    df = quandl.get("BCB/4385", trim_start = "1975-01-01", authtoken = api_key);
    df["Value"] = (df["Value"] - df["Value"][0]) / df["Value"][0] * 100.0; # <-------
    df = df.resample("M").mean();
    df.rename(columns={"Value":"GDP"}, inplace = True);
    df = df["GDP"];
    return df;
    

In [6]:
def us_unemployment():
    df = quandl.get("ECPI/JOB_G", trim_start = "1975-01-01", authtoken = api_key);
    df["Unemployment Rate"] = (df["Unemployment Rate"] - df["Unemployment Rate"][0]) / df["Unemployment Rate"][0] * 100.0; # <-------
    df = df.resample("1D").mean();
    df = df.resample("M").mean();
    return df;

In [7]:
sp500 = sp500_data();
US_GDP = gdp_data();
US_uneployment = us_unemployment();

m30 = mortgage_30y_resampled();
HPI_data = pd.read_pickle("./data/fiddy_states.pickle");
HPI_bench = HPI_Benchmark();

In [8]:
HPI = HPI_data.join([HPI_bench, m30, US_uneployment, US_GDP, sp500]);
print(HPI.head());
print(HPI.corr().head());
# we have nans!!

                  AL        AK        AZ        AR        CA        CO  \
Date                                                                     
1975-01-31  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1975-02-28  0.626500  1.485775  1.688504  0.846192  0.356177  1.159639   
1975-03-31  1.358575  3.006473  3.261346  1.581956  1.575690  2.299449   
1975-04-30  2.254726  4.593530  4.475810  2.183669  3.573196  3.359028   
1975-05-31  3.107829  6.327600  5.139617  2.786248  5.241395  4.226895   

                  CT        DE         FL        GA    ...            VA  \
Date                                                   ...                 
1975-01-31  0.000000  0.000000   0.000000  0.000000    ...      0.000000   
1975-02-28  2.123926  0.142451   3.938796 -0.902841    ...      0.987288   
1975-03-31  3.719898  0.387918   9.798243 -1.282758    ...      1.707474   
1975-04-30  4.616778  0.891619  16.974819 -1.068371    ...      2.238392   
1975-05-31  4.901787  1.7

In [9]:
HPI.dropna(inplace = True);
print(HPI.head());
print(HPI.corr().head());

                   AL         AK          AZ         AR          CA  \
Date                                                                  
1990-01-31  97.377466  62.960664  125.925384  90.013079  407.639219   
1990-02-28  97.306825  64.745987  125.954481  90.292939  409.393746   
1990-03-31  97.640926  68.843816  125.677184  90.749096  412.358848   
1990-04-30  98.347335  75.545290  125.537118  91.190719  414.727667   
1990-05-31  99.135672  84.269489  125.820526  91.693828  415.785301   

                    CO          CT          DE          FL          GA  \
Date                                                                     
1990-01-31  128.607770  289.087700  202.214111  133.668588  116.794322   
1990-02-28  128.817810  286.729447  203.585164  133.302517  116.293285   
1990-03-31  129.016696  283.918663  203.903611  133.217207  115.513095   
1990-04-30  129.123827  280.649292  203.811564  133.568341  115.303254   
1990-05-31  129.785821  277.504604  203.813374  134.063840

In [10]:
HPI.to_pickle("./data/HPI.pickle");

In [11]:
housing_data = pd.read_pickle("./data/HPI.pickle");
housing_data = housing_data.pct_change();
print(housing_data.head());

                  AL        AK        AZ        AR        CA        CO  \
Date                                                                     
1990-01-31       NaN       NaN       NaN       NaN       NaN       NaN   
1990-02-28 -0.000725  0.028356  0.000231  0.003109  0.004304  0.001633   
1990-03-31  0.003433  0.063291 -0.002202  0.005052  0.007243  0.001544   
1990-04-30  0.007235  0.097343 -0.001114  0.004866  0.005745  0.000830   
1990-05-31  0.008016  0.115483  0.002258  0.005517  0.002550  0.005127   

                  CT        DE        FL        GA    ...           VA  \
Date                                                  ...                
1990-01-31       NaN       NaN       NaN       NaN    ...          NaN   
1990-02-28 -0.008158  0.006780 -0.002739 -0.004290    ...    -0.001692   
1990-03-31 -0.009803  0.001564 -0.000640 -0.006709    ...     0.001023   
1990-04-30 -0.011515 -0.000451  0.002636 -0.001817    ...     0.005560   
1990-05-31 -0.011205  0.000009  0.003

In [12]:
housing_data.replace([np.inf, -np.inf], np.nan, inplace = True);
housing_data.dropna(inplace = True);
print(housing_data.head());

                  AL        AK        AZ        AR        CA        CO  \
Date                                                                     
1990-03-31  0.003433  0.063291 -0.002202  0.005052  0.007243  0.001544   
1990-04-30  0.007235  0.097343 -0.001114  0.004866  0.005745  0.000830   
1990-05-31  0.008016  0.115483  0.002258  0.005517  0.002550  0.005127   
1990-06-30  0.004616  0.103961  0.003969  0.006417  0.003595  0.007326   
1990-07-31  0.000083  0.069205  0.001866  0.006427  0.004748  0.003203   

                  CT        DE        FL        GA    ...           VA  \
Date                                                  ...                
1990-03-31 -0.009803  0.001564 -0.000640 -0.006709    ...     0.001023   
1990-04-30 -0.011515 -0.000451  0.002636 -0.001817    ...     0.005560   
1990-05-31 -0.011205  0.000009  0.003710  0.003266    ...     0.005750   
1990-06-30 -0.007019 -0.000411  0.003081  0.002858    ...     0.003002   
1990-07-31 -0.003224 -0.003613  0.002

In [13]:
housing_data["US_HPI_future"] = housing_data["US"].shift(-1);
print(housing_data[["US_HPI_future", "US"]].head());

            US_HPI_future        US
Date                               
1990-03-31       0.005182  0.004323
1990-04-30       0.005244  0.005182
1990-05-31       0.005103  0.005244
1990-06-30       0.003712  0.005103
1990-07-31       0.000489  0.003712


In [14]:
def create_labels(cur_hpi, fut_hpi):
    if fut_hpi > cur_hpi:
        return 1;
    else:
        return 0;

housing_data["label"] = list(map(create_labels, housing_data["US"], housing_data["US_HPI_future"])); # wow
#pd.Series.map may be useful also
print(housing_data[["US_HPI_future", "US", "label"]].head());

            US_HPI_future        US  label
Date                                      
1990-03-31       0.005182  0.004323      1
1990-04-30       0.005244  0.005182      1
1990-05-31       0.005103  0.005244      0
1990-06-30       0.003712  0.005103      0
1990-07-31       0.000489  0.003712      0


In [15]:
def moving_average(values):
    return mean(values);

housing_data["ma_apply_example"] = housing_data["M30"].rolling(window = 10).apply(moving_average);
print(housing_data[["M30", "ma_apply_example"]]);

                 M30  ma_apply_example
Date                                  
1990-03-31  0.090909               NaN
1990-04-30  0.119048               NaN
1990-05-31  0.117021               NaN
1990-06-30 -0.304762               NaN
1990-07-31 -0.164384               NaN
1990-08-31  0.098361               NaN
1990-09-30  0.119403               NaN
1990-10-31  0.000000               NaN
1990-11-30 -0.226667               NaN
1990-12-31 -0.586207         -0.073728
1991-01-31 -0.125000         -0.095319
1991-02-28 -1.285714         -0.235795
1991-03-31 -2.166667         -0.464164
1991-04-30 -0.142857         -0.447973
1991-05-31 -0.333333         -0.464868
1991-06-30  3.750000         -0.099704
1991-07-31 -0.210526         -0.132697
1991-08-31 -2.266667         -0.359364
1991-09-30  1.210526         -0.215644
1991-10-31  0.357143         -0.121310
1991-11-30  0.263158         -0.082494
1991-12-31  0.291667          0.075244
1992-01-31  0.075269          0.299438
1992-02-29 -0.330000     

# in fact, this notebook starts here

In [16]:
X = np.array(housing_data.drop(["US_HPI_future", "label", "ma_apply_example"], 1));
print(X);
X = preprocessing.scale(X);
print(X);

[[  3.43348369e-03   6.32908563e-02  -2.20155868e-03 ...,   3.57142857e-02
   -2.34375000e-01   3.07904247e-02]
 [  7.23476661e-03   9.73431485e-02  -1.11449223e-03 ...,  -6.89655172e-02
    4.26530612e+00  -1.06996554e-03]
 [  8.01583787e-03   1.15483024e-01   2.25756447e-03 ...,   0.00000000e+00
   -1.09253876e+00   4.50535306e-02]
 ..., 
 [ -1.38395068e-02  -7.53032397e-03   6.90772620e-04 ...,  -1.11111111e-01
    2.01368115e-02   3.02062770e-02]
 [ -6.51605813e-03  -7.08517825e-03   6.67412020e-03 ...,  -2.50000000e-01
    9.48695005e-04   1.68855043e-02]
 [  3.60639587e-04  -5.30536889e-03   7.65538356e-03 ...,  -3.33333333e-01
   -2.21278017e-02   1.46240254e-02]]
[[ 0.05008063  4.0429684  -0.3829891  ...,  0.1928799  -0.39143479
   0.59380827]
 [ 0.64096895  6.46761673 -0.30118641 ..., -0.34198895  4.28569477
  -0.18863566]
 [ 0.76238214  7.75924253 -0.04743625 ...,  0.01039524 -1.28344091
   0.944089  ]
 ..., 
 [-2.63490981 -0.99975986 -0.16533868 ..., -0.55733484 -0.1268861
 

In [17]:
y = np.array(housing_data["label"]);
print(y);

[1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 0 1 1 0 0 0 1 1 1 0 0 0 0 0 1 1 1
 1 0 0 0 0 0 1 0 1 1 1 1 1 0 0 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 0
 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1 0
 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0
 0 0 0 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0
 0 0 0 0 1 1 1 1 0 0 0 0 0 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0
 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1 0 0 0 0 0
 1 0 0]


In [20]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size = 0.2);
clf = svm.SVC(kernel = "linear");
clf.fit(X_train, y_train);

print(clf.score(X_test, y_test));

0.811320754717
